In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow import keras

In [2]:
BATCH_SIZE = 512
SHUFFLE_BUFFER_SIZE = 10000

In [3]:
train_data, val_data, test_data = tfds.load(
    name='imdb_reviews', 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

In [4]:
train_ds = train_data\
    .shuffle(SHUFFLE_BUFFER_SIZE)\
    .batch(BATCH_SIZE)

val_ds = val_data.batch(BATCH_SIZE)

test_ds = test_data.batch(BATCH_SIZE)

In [5]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

In [6]:
model = keras.Sequential([
    hub_layer,
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(
    optimizer='adam',
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [8]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    verbose=1
)

Epoch 1/10
30/30 [==============================] - 3s 94ms/step - loss: 0.6869 - accuracy: 0.4973 - val_loss: 0.6535 - val_accuracy: 0.5206
Epoch 2/10
30/30 [==============================] - 3s 84ms/step - loss: 0.6131 - accuracy: 0.5797 - val_loss: 0.5819 - val_accuracy: 0.6448
Epoch 3/10
30/30 [==============================] - 3s 85ms/step - loss: 0.5139 - accuracy: 0.7284 - val_loss: 0.4886 - val_accuracy: 0.7458
Epoch 4/10
30/30 [==============================] - 3s 84ms/step - loss: 0.3980 - accuracy: 0.8304 - val_loss: 0.4039 - val_accuracy: 0.8275
Epoch 5/10
30/30 [==============================] - 3s 84ms/step - loss: 0.2965 - accuracy: 0.8903 - val_loss: 0.3503 - val_accuracy: 0.8520
Epoch 6/10
30/30 [==============================] - 3s 85ms/step - loss: 0.2229 - accuracy: 0.9234 - val_loss: 0.3215 - val_accuracy: 0.8613
Epoch 7/10
30/30 [==============================] - 3s 84ms/step - loss: 0.1680 - accuracy: 0.9465 - val_loss: 0.3077 - val_accuracy: 0.8667
Epoch 8/10
30

In [9]:
result = model.evaluate(test_ds, verbose=2)

49/49 - 1s - loss: 0.3352 - accuracy: 0.8575


In [10]:
new_data = "There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come."
logits = model.predict([new_data])
prediction = tf.sigmoid(logits)
prediction

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.923278]], dtype=float32)>